Implemented hit selection, events scatter, ion heatmap, electrons time of flight and ions time of flight

### Imports and functions

In [ ]:
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def events_selection_plots(runs,lower_threshold,upper_threshold):
    'Runs functions events_selection, heatmap, e_tof, ion_tof'
    
    selected_dfevent, selected_dfpulse, selected_etof = events_selection(runs,lower_threshold,upper_threshold)
    heatmap(selected_dfevent)
    e_tof(selected_etof)
    ion_tof(selected_dfevent)
    
    return selected_dfevent, selected_dfpulse, selected_etof



def events_selection(runs,lower_threshold,upper_threshold):
    'Reads one run from h5 files'
    'Makes a pulse selection based on the number of events per pulse between the defined thresholds'
    
    dfevent, dfpulse, etof, pnccd = read(runs)
    
    selected_dfpulse = dfpulse[lower_threshold < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold]
    selected_dfevent = dfevent[dfevent.pulseId.isin(selected_dfpulse.pulseId)]
    selected_etof = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse.pulseId))
    
    plt.figure()
    plt.scatter(dfpulse.pulseId,dfpulse.nevents_pulse,label='All pulses')
    plt.scatter(selected_dfpulse.pulseId,selected_dfpulse.nevents_pulse,c='r',label='Selected pulses')
    plt.xlabel('Pulse ID')
    plt.ylabel('Number of events per pulse')
    plt.legend()
    plt.title('Events per pulse with respect to pulse ID')
    plt.show()
    
    return selected_dfevent, selected_dfpulse, selected_etof



def events_backgrd(runs,lower_backgrd_threshold,upper_backgrd_threshold):
    'Reads one or multiple runs from h5 files'
    'Makes a pulse selection based on the number of events per pulse between the background defined thresholds'
    'If multiple runs are passed, will merge the runs, once hit selected'
    
    dfevent, dfpulse, etof, pnccd = read(runs)
    
    backgrd_dfpulse = dfpulse[lower_backgrd_threshold < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_backgrd_threshold]
    backgrd_dfevent = dfevent[dfevent.pulseId.isin(backgrd_dfpulse.pulseId)]
    backgrd_etof = etof.sel(pulseId=etof.coords['pulseId'].isin(backgrd_dfpulse.pulseId))
    
    plt.figure()
    plt.scatter(dfpulse.pulseId,dfpulse.nevents_pulse,label='All pulses')
    plt.scatter(backgrd_dfpulse.pulseId,backgrd_dfpulse.nevents_pulse,c='g',label='Backgrounds pulses')
    plt.xlabel('Pulse ID')
    plt.ylabel('Number of events per pulse')
    plt.legend()
    plt.title('Events per pulse with respect to pulse ID')
    plt.show()
    
    return backgrd_dfevent, backgrd_dfpulse, backgrd_etof



def read(runid):
    'Read the preprocessed data of run with ID runid saved in the h5 file with a corresponding name'
    'Outputs dataframes per event, per pulse, and xarrays etof, pnccd in that order'
    
    filename = '../preprocess/datarun' + str(runid) + '.h5'
    
    dfevent = pd.read_hdf(filename, 'dfevent')
    dfpulse = pd.read_hdf(filename, 'dfpulse')
    
    etof = xr.open_dataarray(filename, group="etof")
    pnccd = xr.open_dataarray(filename, group="pnccd")
    
    return dfevent, dfpulse, etof, pnccd



def heatmap(dfevent):
    'Creates heatmap of the ions hits, based on a dfevent dataframe'
    
    counts_df = dfevent.groupby(['x', 'y']).size().reset_index(name='count')
    heatmap_data = counts_df.pivot(index='y', columns='x', values='count')
    
    plt.figure()
    sns.heatmap(heatmap_data, cmap='viridis',cbar_kws={'label': 'Number of events'})
    plt.title('Ion heatmap')
    plt.show()
    
    
    
def e_tof(etof):
    'Plots electron time of flight data using etof xarray data'
    
    TIME_BETWEEN_PULSES = 3.54462e-6
    CHANNELS_PER_PULSE = 14080
    channel_time = TIME_BETWEEN_PULSES/CHANNELS_PER_PULSE
    
    xaxis = np.arange(14080)*channel_time
    avg_selected_etof = -np.mean(etof, axis=0)
    
    plt.figure()
    plt.plot(xaxis,avg_selected_etof/max(avg_selected_etof))
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Normalized signal')
    plt.title('Electrons time of flight')
    plt.show()
    
    
    
def ion_tof(dfevent):
    'Plots ion time of flight data using dfevent dataframe'
    
    hist, bin_edges = np.histogram(dfevent.tof, bins=250000)
    hist1 = hist[:1500]
    bin_edges1 = bin_edges[:1501]
    
    plt.figure()
    plt.plot(bin_edges1[:-1], hist1)
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Number of hits per bin')
    plt.title('Ions time of flight')
    plt.show()
    
    
    
